In [1]:
import logging
import time
import pandas as pd
from datetime import datetime
from iqoptionapi.stable_api import IQ_Option

In [2]:
list_asset = []
num_candles = 120
balance = 0
balance_win = 0

In [3]:
print("Conectando...")
Iq = IQ_Option("gggeeelt66612@gmail.com","vSsUK6wM5DW8mLWfZ")
status, reason = Iq.connect()
print('##### Primeira tentativa #####')
print('Status:', status)
balance = Iq.get_balance()
print('Balance:', balance)

Conectando...
##### Primeira tentativa #####
Status: True
Balance: 9720


In [4]:
def get_assets_open():
    global list_asset
    list_asset.clear()
    all_asset=Iq.get_all_open_time()
    for type_asset, data in all_asset.items():
        for Asset,value in data.items():
            if type_asset == "turbo" and value["open"]:
               list_asset.append(Asset)
    print("get assets open")
get_assets_open()

get assets open


In [5]:
def get_position(asset):
    global num_candles
    candles = Iq.get_candles(asset,60,num_candles,time.time())
    df = pd.DataFrame(candles)
    df['color'] = ['g' if close > open else 'r' for open, close in zip(df['open'], df['close'])]
    count_r = df['color'].value_counts()['r']
    count_g = df['color'].value_counts()['g']
    action = None
    prediction = None
    percentage = 0
    #print("{} - Count R: {} Count G: {}".format(asset,count_r,count_g))
    if count_r > count_g :
        percentage = (count_g / num_candles) * 100
        action = "call"
        print("{} - Percentage call {}%".format(asset,percentage.round(2)))
    elif count_g > count_r :
        percentage = (count_r / num_candles) * 100
        action = "put"
        print("{} - Percentage put {}%".format(asset ,percentage.round(2)))
    else:
        print("{} - Percentage equal".format(asset))
    
    if action is not None and percentage < 35:
        last = df.tail(3)
        try:
            count_r = last['color'].value_counts()['r']
        except Exception as e:
            count_r = 0
        if count_r == 3 or count_r == 2:
            prediction = "call"
        elif count_r == 1 or count_r == 0:
            prediction = "put"
        else:
            prediction = None
    else:
        action = None
    
    if prediction == action:
        return {"action": action, "percentage": percentage}
    else:
        return {"action": None, "percentage": percentage}
    
for asset in list_asset:
    get_position(asset)

EURGBP-OTC - Percentage call 35.83%
EURUSD-OTC - Percentage call 47.5%
GBPUSD-OTC - Percentage call 46.67%
EURJPY-OTC - Percentage call 46.67%
AUDCAD-OTC - Percentage put 49.17%
NZDUSD-OTC - Percentage put 48.33%
USDCHF-OTC - Percentage call 48.33%


In [6]:
def buy(asset, action):
    check,id=Iq.buy(10,asset,action,1)
    if check:
        print("!buy!")
        balance_win = Iq.get_balance() - balance
        print('Balance:', balance)
        print('Balance Win:', balance_win)
    else:
        print("buy fail")
        get_assets_open()
        
        

In [7]:
def is_time ():
    minutos = float(((datetime.now()).strftime('%M.%S'))[1:])
    # segundos = float(((datetime.now()).strftime('%S')))
    # print('minutos son: \n', minutos)
    # print('segundos son: \n', segundos)
    enter = False
    if minutos == 4.58 or minutos == 9.58:
        enter = True
    # if segundos == 28.00 or segundos == 58.0:
    # if segundos == 58.0:
    #     enter = True
    return enter


In [8]:
while True:
    if is_time():
        print("check time")
        for asset in list_asset:
            result = get_position(asset)
            if result["action"] is not None:
                buy(asset, result["action"])
                break

check time
EURGBP-OTC - Percentage call 35.83%
EURUSD-OTC - Percentage call 47.5%
GBPUSD-OTC - Percentage call 48.33%
EURJPY-OTC - Percentage call 45.83%
AUDCAD-OTC - Percentage put 47.5%
NZDUSD-OTC - Percentage put 48.33%
USDCHF-OTC - Percentage equal
check time
EURGBP-OTC - Percentage call 34.17%
EURUSD-OTC - Percentage call 45.0%
GBPUSD-OTC - Percentage call 48.33%
EURJPY-OTC - Percentage call 46.67%
AUDCAD-OTC - Percentage put 45.83%
NZDUSD-OTC - Percentage put 49.17%
USDCHF-OTC - Percentage equal
check time
EURGBP-OTC - Percentage call 34.17%
!buy!
Balance: 9720
Balance Win: -10
check time
EURGBP-OTC - Percentage call 30.83%
!buy!
Balance: 9720
Balance Win: -70
check time
EURGBP-OTC - Percentage call 29.17%


ERROR:root:**warning** buy late 5 sec


buy fail
get assets open
check time
EURGBP-OTC - Percentage call 31.67%
AUDCAD-OTC - Percentage put 45.83%
GBPUSD-OTC - Percentage call 45.83%
NZDUSD-OTC - Percentage call 49.17%
EURJPY-OTC - Percentage call 48.33%
USDCHF-OTC - Percentage put 48.33%
EURUSD-OTC - Percentage call 43.33%
check time
EURGBP-OTC - Percentage call 30.0%
!buy!
Balance: 9720
Balance Win: -44
check time
EURGBP-OTC - Percentage call 28.33%
!buy!
Balance: 9720
Balance Win: -1.5
check time
EURGBP-OTC - Percentage call 29.17%
AUDCAD-OTC - Percentage equal
GBPUSD-OTC - Percentage call 48.33%
NZDUSD-OTC - Percentage call 48.33%
EURJPY-OTC - Percentage call 46.67%
USDCHF-OTC - Percentage call 48.33%
EURUSD-OTC - Percentage call 45.0%
check time
EURGBP-OTC - Percentage call 30.0%
AUDCAD-OTC - Percentage call 49.17%
GBPUSD-OTC - Percentage call 47.5%
NZDUSD-OTC - Percentage call 47.5%
EURJPY-OTC - Percentage call 45.83%
USDCHF-OTC - Percentage call 48.33%
EURUSD-OTC - Percentage call 43.33%
check time
EURGBP-OTC - Percen

ERROR:iqoptionapi.ws.client:Connection is already closed.
ERROR:root:**error** get_candles need reconnect
ERROR:iqoptionapi.ws.client:[Errno 11001] getaddrinfo failed


check time


JSONDecodeError: Expecting value: line 1 column 2 (char 1)